# Power Analysis

A power analysis is similar to that of exploratory data analysis, the power analysis should be done thoroughly and explored through many different elements. For example, power should be evaluated across a variety of realistic data conditions. 

## What is Power?

Power is the ability to detect an effect of interest. Put into statistical terms, this could be phrased as detecting a non-zero effect or rejecting the statistical null hypothesis (or more accurately providing evidence that the statistical null hypothesis is unlikely to be true with the sample data). 

Power is typically used for study planning and is often required by grant funding agencies. As suggested in the title, this book advocates for estimating power by simulation, but this is not the only framework for estimating power. Depending on the method, there may be closed form solutions that provide an alternative to estimating power by simulation. As discussed in Chapter 1, the primary benefits of power by simulation include: 

When defining power, it is important to first define the statistical hypotheses that are framing the question(s) of interest. Although these will likely vary depending on the data collection method and statistical model fitted, the null and alternative hypotheses will typically follow this general framework when the analysis can be framed as a linear regression:

$$
H_{0}: \beta_{t} = 0
$$

$$ 
H_{A}: \beta_{t} \neq 0
$$

where $H_{0}$ is the null hypothesis representing that the regression parameter ($\beta_{t}$) is equal to 0 in the population. The null hypothesis is the statistical hypothesis that the p-value is interpreted with respect to in frequentist statistics. When evaluating power, the second hypothesis, $H_{A}$ or the alternative hypothesis, is of more importance. This hypothesis represents the effect of interest or the distribution of the effect. More generally, this hypothesis is the hypothesis that would be true when the parameter of interest is not zero in the population, a common (but not always true) assumption when evaluating power. 

If you have taken a statistics course before, you may have been shown the two by two table. This table shows the possible decisions that can be made when using the hypothesis testing framework above. 

|   | $H_{0}$ true | $H_{0}$ FALSE |
|:--|-------------:|--------------:|
| Reject $H_{0}$ | Type I Error | Correct (power) | 
| Retain $H_{0}$ | Correct      | Type II Error |

Let's assume the following model, this is essentially a two-sample t-test model where I am assuming that $group_{i}$ below represents two values. 

$$
Y_{i} = \beta_{0} + \beta_{1} group_{i} + \epsilon_{i} 
$$

Power can be evaluated in a few different ways, one is through closed form equations that estimation power. A second way is to use power by simulation, which can be more accurate as estimating power through the closed form equations makes assumptions about the model. 

One of the assumptions needed to be made is the size of the effect that will be estimated, these are typically done through the effect size metric. In the two-sample design, in many instances this represents the standardized mean difference. 

In [ ]:
library(tidyverse)

n <- seq(4, 1000, 2)
power <- sapply(seq_along(n), function(i) 
  power.t.test(n = n[i], delta = .15, sd = 1, type = 'two.sample')$power)

power_df <- data.frame(
  n = n,
  power = power
)

ggplot(power_df, aes(x = n, y = power)) + 
  geom_line(size = 2) + 
  geom_hline(yintercept = 0.8, linetype = 2, color = 'gray30') + 
  geom_vline(xintercept = 700, linetype = 2, color = 'gray30') +
  scale_x_continuous("Sample Size", breaks = seq(0, 1000, 200)) + 
  scale_y_continuous("Power", breaks = seq(0, 1, .2)) +
  theme_bw(base_size = 14)

## Power Curves

Power curves can be useful to show how power may change based on different values. Above, a power curve was plotted across sample sizes, but the following code estimates power for different effect size values.

In [ ]:
effect_sizes <- c(.10, .15, .25)
conditions <- expand.grid(n = n, effect_sizes = effect_sizes)

power_curve <- sapply(seq_len(nrow(conditions)), function(i) 
  power.t.test(n = conditions[i, 'n'], 
               delta = conditions[i, 'effect_sizes'], 
               sd = 1, type = 'two.sample')$power)

power_curve_df <- bind_cols(
  conditions, 
  power = power_curve
)

ggplot(power_curve_df, aes(x = n, y = power)) + 
  geom_line(aes(color = factor(effect_sizes)), size = 2) + 
  geom_hline(yintercept = 0.8, linetype = 2, color = 'gray30') + 
  scale_x_continuous("Sample Size", breaks = seq(0, 1000, 200)) + 
  scale_y_continuous("Power", breaks = seq(0, 1, .2)) +
  scale_color_discrete("Effect Size") +
  theme_bw(base_size = 14)

## Power by simulation

Power by simulation is similar, but it does not need to make as many assumptions about the data to be collected. As long as you are able to simulate the data, different conditions can be evaluated. This is particularly true for more complicated data situations.

In [ ]:
library(simglm)
library(future)

plan(multicore)

simulation_arguments <- list(
  formula = y ~ 1 + group, 
  fixed = list(group = list(var_type = 'factor', 
                            levels = c('red', 'blue'))),
  error = list(variance = 1),
  reg_weights = c(0, .15),
  replications = 10000,
  model_fit = list(formula = y ~ 1 + group,
                   model_function = 'lm'),
  power = list(
    dist = 'qt',
    alpha = .05
  ),
  extract_coefficients = TRUE,
  vary_arguments = list(
    sample_size = seq(20, 2000, 20) 
  )
)

model_results <- replicate_simulation(simulation_arguments) 

power_results <- model_results %>%
  compute_statistics(simulation_arguments, power = TRUE,
                     type_1_error = FALSE, precision = TRUE)

power_results <- power_results %>%
  ungroup() %>%
  mutate(sample_size = as.numeric(as.character(sample_size))) %>%
  arrange(sample_size) %>%
  filter(term == 'group')

ggplot(power_results, aes(x = sample_size, y = power)) + 
  geom_point(size = 1.5, color = 'gray40') + 
  geom_hline(yintercept = 0.8, linetype = 2, color = 'gray30') + 
  geom_smooth(linetype = 1, size = 1, se = FALSE) +
  geom_vline(xintercept = min(filter(power_results, power >= 0.8)$power), 
             linetype = 2, color = 'gray30') +
  scale_x_continuous("Sample Size", breaks = seq(0, 2000, 200)) + 
  scale_y_continuous("Power", breaks = seq(0, 1, .2)) +
  theme_bw(base_size = 14)

In [ ]:
model_results_df <- bind_rows(model_results) %>%
  filter(term == 'group') %>%
  mutate(sample_size = as.numeric(as.character(sample_size)))

ggplot(model_results_df, aes(x = sample_size, y = estimate)) + 
  geom_boxplot(aes(group = sample_size), outlier.alpha = 0.01) +
  scale_x_continuous("Sample Size", breaks = seq(0, 2000, 200)) +
  theme_bw(base_size = 14) +
  geom_hline(yintercept = .15, linetype = 2, size = 2, color = 'gray20') +
  coord_flip()

### Add Heterogeneity

In [ ]:
simulation_arguments <- list(
  formula = y ~ 1 + group,
  fixed = list(group = list(var_type = 'factor', 
                            levels = c('red', 'blue'))),
  error = list(variance = 1),
  heterogeneity = list(variable = 'group',
                       variance = c(1, 8)),
  reg_weights = c(0, .15),
  replications = 10000,
  model_fit = list(formula = y ~ 1 + group, 
                   model_function = 'lm'),
  power = list(
    dist = 'qnorm',
    alpha = .05
  ),
  extract_coefficients = TRUE,
  vary_arguments = list(
    sample_size = seq(20, 2000, 20) 
  )
)

model_results_h <- replicate_simulation(simulation_arguments)

power_results_h <- model_results_h %>%
  compute_statistics(simulation_arguments, power = TRUE,
                     type_1_error = FALSE, precision = TRUE) %>%
  ungroup() %>%
  mutate(sample_size = as.numeric(as.character(sample_size)),
         heterogeneity = TRUE) %>%
  arrange(sample_size) %>%
  filter(term == 'group')

power_results <- power_results %>%
  mutate(heterogeneity = FALSE)

power_results_combined <- bind_rows(power_results, 
                                    power_results_h)

ggplot(power_results_combined, aes(x = sample_size, y = power, 
                          group = heterogeneity)) + 
  geom_point(aes(shape = heterogeneity), size = 1.5, color = 'gray40') + 
  geom_hline(yintercept = 0.8, linetype = 2, color = 'gray30') + 
  geom_smooth(aes(linetype = heterogeneity), size = 1, se = FALSE) +
  scale_x_continuous("Sample Size", breaks = seq(0, 2000, 200)) + 
  scale_y_continuous("Power", breaks = seq(0, 1, .2)) +
  theme_bw(base_size = 14)